##Experiment

In [ ]:
!pip install cvxpy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 5.2 MB/s 
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [ ]:
import numpy as np
import cvxpy as cp
import numpy.linalg as linalg
from sklearn.linear_model import Lasso
from copy import deepcopy, copy 

In [1]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

In [ ]:
np.random.seed(42)

In [ ]:
def get_psd(X):
    n = X.shape[0]
    A = cp.Variable((n,n),symmetric=True)
    const = [A >> 0]
    cost = cp.norm(A-X,"fro")
    prob = cp.Problem(cp.Minimize(cost), const)
    prob.solve()
    return  A.value

def get_obj_fct(lasso):
    def obj_fct_n(Theta, Y , eta , lmbd, n ):
        #This needs fiving
        obj_fct = 0.0
        for nn in range(n):
            for t in range(Y[nn].shape[0]):
                y = Y[nn][t]
                lasso.fit(Theta.T, y ) # involves solving a LASSO problem
                hj = lasso.coef_
                obj_fct += 0.5*linalg.norm(y - hj@Theta )**2 + lmbd*linalg.norm( hj, 1)
        return (obj_fct + 0.5*eta* linalg.norm( Theta )**2)
    return obj_fct_n

def quant(x, s):
    return x
    # return (1/s) * np.sign(x) * linalg.norm(x) * np.floor(s* np.abs(x)/(linalg.norm(x))* 
    #                                                       np.random.uniform(size=x.shape))

## The Code

In [ ]:
d = 15
m = 10
n = 20 # number of workers!
T = 50 # T is the total no of samples

# pre-generate the model
Theta_true = np.random.normal(size=(d,m))
Y , X_true = [], []

#TODO
#This needs to be reviewed.
for nn in range(n):
    X_true.append(np.random.uniform(size=(T,d))* np.random.binomial(1, 0.2,size=(T,d)))
    Y.append(X_true[nn]@Theta_true + 0.0001*np.random.normal(size=(T,m)))

# combine the dataset and form the dataset with "homogeneous" data
Y_homo = [];
Y_concat = np.vstack(Y)

#I dont understand this
for i in range(n):
    #may neeed a deep copy here
    Y_homo.append(Y_concat)

In [ ]:
eta = 0.1;
lmbd = 0.05; # require values for lambda
lasso = Lasso(alpha=lmbd, fit_intercept=False, max_iter=5000)
Theta_init = np.random.normal(size=(d,m))

obj_fct_n = get_obj_fct(lasso)

In [ ]:
# Run the Fed-DL Algorithm
homo_switch = 0; # 0 - hete data, 1 - homo. data
rep_times = 10; # no. of repetition to run the experiment

# parameters for the algorithms 
BatchSize = 20; # batch size
Participate = 5; # no of active workers 
alpha = 0.01; # alpha parameter in the algorithm
squant = 15; # quantization level
IterNum = 5000; # no. of iterations to run

p = (Participate/n); 

# variable to save objective values
obj_sto_all = []

for rep in range(rep_times):

    Theta = copy(Theta_init)
    V_SS1 = []; 
    V_SS2 = []; 
    Msg_SS1 = []; 
    Msg_SS2 = [];
    for nn in range(n):
        V_SS1.append(np.zeros((d,d)))
        V_SS2.append(np.zeros((d,m)))
        Msg_SS1.append(np.zeros((d,d)))
        Msg_SS2.append(np.zeros((d,m)))

    Va_SS1, Ha_SS1, Sa_SS1 = np.zeros((d,d)), np.zeros((d,d)),np.zeros((d,d)) 
    Va_SS2, Ha_SS2, Sa_SS2 = np.zeros((d,m)),np.zeros((d,m)),np.zeros((d,m))
    obj_sto = []

    print("\n rep = ", rep, " ")


    for t in range(IterNum):
        if t%10 == 0:
            print( t , ", obj = ")
            obj_sto.append(obj_fct_n( Theta, Y, eta, lmbd, n ))
            print( obj_sto[-1], " " );
        

        # select the workers to participate
        idx_PP = np.random.permutation(n)[:Participate]
        for nn in idx_PP:
            # we are in agent nn
            if homo_switch == 0:
                idx_batch = np.random.permutation(T)[:BatchSize]
                #This needs review.....
                yy = Y[nn][idx_batch]
            else:
                idx_batch = np.random.permutation(n*T)[:BatchSize]
                yy = Y_homo[nn][idx_batch] 
            
            SS1 = np.zeros((d,d))
            SS2 = np.zeros((d,m));
            for b in range(BatchSize):
                fitted = lasso.fit(Theta.T,yy[b])
                hj = fitted.coef_
                # construct the test statistics
                SS1 += np.outer(hj,hj)
                SS2 += np.outer(hj,yy[b])

            ################################
            # construct the message
            Delta1 = SS1/BatchSize - Sa_SS1 - V_SS1[nn];
            Delta2 = SS2/BatchSize - Sa_SS2 - V_SS2[nn];
            V_SS1[nn] = V_SS1[nn] + (alpha/p)*quant( Delta1, squant )
            V_SS2[nn] = V_SS2[nn] + (alpha/p)*quant( Delta2, squant )
            Msg_SS1[nn] = quant(Delta1, squant)
            Msg_SS2[nn] = quant(Delta2, squant); 
        

        # Server Update
        step_size = 0.1 / np.sqrt(t+0.1)
        tmp_SS1 = sum([Msg_SS1[i] for i in idx_PP])
        tmp_SS1 = (tmp_SS1+tmp_SS1.T)/2
        Ha_SS1 = Va_SS1 + (1/(n*p))*tmp_SS1
        Ha_SS2 = Va_SS2 + (1/(n*p))*sum( Msg_SS2[i] for i in idx_PP)

        # projected version!
        Sa_SS1 = Sa_SS1 + step_size*Ha_SS1;
        #This should be the biggest problem....
        Sa_SS1_Proj = get_psd(0.5*(Sa_SS1+Sa_SS1.T))
        Sa_SS1 = Sa_SS1_Proj

        Sa_SS2 = Sa_SS2 + step_size*Ha_SS2;
        Va_SS1 = Va_SS1 + (alpha/(n*p))*tmp_SS1
        Va_SS2 = Va_SS2 + (alpha/(n*p))*sum([Msg_SS2[i]for i in idx_PP] )

        # compute the new Theta
        Theta =   (Sa_SS1 + eta*np.eye(d))**(-1) @ Sa_SS2
        # print(np.isnan(Theta).any())
        # print(linalg.eigvals(Sa_SS1))
        # print(Theta.shape)
    obj_sto_all.append(obj_sto)